selecionar o clock para externo e configurar o awg para 10 MHz

selecionar dual channel with markers

selecionar sampling rate -> 64 GSamples/s

colocar o canal 2 para usar o memória externa -> dividir a memória para dois canais (isso divide o sampling rate)

habilitar todos os canais


# Setting TCP/IP Connection with Sockets

In [55]:
from socket import socket, AF_INET, SOCK_STREAM
from time import sleep
import numpy as np
import matplotlib.pyplot as plt

In [6]:
session = socket(AF_INET, SOCK_STREAM)

In [9]:
session.connect(('169.254.101.100',5025))

In [45]:
session.settimeout(4)

In [58]:

def write(cmd):
    session.sendall(str.encode(cmd+'\n'))

def query(cmd):
    write(cmd)
    sleep(0.1)
    return session.recv(4096).decode()

def getError():
    return query('SYSTEM:ERR?')


In [59]:
query('*IDN?')

'Keysight Technologies,M8195A,MY55A00568,4.1.0.0-2\n'

In [60]:
getError()

'0,"No error"\n'

# Setting AWG

### Clock

In [65]:
write(':rosc:sour ext')

In [66]:
write(':rosc:freq 10e6')

### Sampling Rate

In [68]:
write(':FREQ:RAST 64e9')

### Dual Channel With Marker

In [69]:
write(':inst:dacm dcm')

### Divide Memory (Sampling Rate also)

In [70]:
write(':INST:MEM:EXT:RDIV DIV2')

### Set Channel 2 to External Memory

In [71]:
write(':TRAC2:MMOD EXT')

### Enable all Channels

In [74]:
write(':outp1 on')
write(':outp2 on')
write(':outp3 on')
write(':outp4 on')

### Set offset Voltage and Amplitude Voltage

In [80]:
write(':volt1 1')
write(':volt2 1')
write(':volt3 1')
write(':volt4 1')

write(':volt3:offs 0.5')
write(':volt4:offs 0.5')

# Loading data to AWG

In [218]:
def start():
    write(':INIT:IMM')

def stop():
    write(':abor')

def load(channel,data, offset = 0 ):
    tamanho = len(data)
    n = int(np.log10(tamanho)+1)
    tag = '#'+str(n) + str(tamanho)
    
    cmd = ':trac{}:data 1, {}, '.format(channel,offset)+tag
    
    payload = cmd.encode() + bytes(data) + '\n'.encode()
    
    session.sendall(payload)


### para baixar os dados

In [ ]:
write(':trac2:block? 1,0, 4096')

In [ ]:
session.recv(4096)

### Gerar os dados para canal 2

In [50]:
length = 10e-9
x = np.arange(0,length,1/32e9)

sigma = 0.2 * length
y = 127*np.exp(-(x-length/2)**2/sigma**2)

padding = np.zeros(192)
y = np.append(y,padding)

data2 = np.array(yWMarkers,dtype=np.int8)

### Gerar os dados para canal 1 com markers

In [51]:
length = 10e-9
x = np.arange(0,length,1/32e9)

sigma = 0.2 * length
y = 127*np.exp(-(x-length/2)**2/sigma**2)

padding = np.zeros(192)
y = np.append(y,padding)

markers = 2*np.ones(len(y))

yWMarkers =  np.array(list(zip(y,markers))).flatten()

data1 = np.array(yWMarkers,dtype=np.int8)

### carregar os dados no AWG

In [328]:
load(2,data2,0) # colocar o dado no canal 1 no offset 0

In [ ]:
load(2,data1,512) # colocar o dado no canal 1 no offset 512

In [320]:
getError()

'-300,"Device-specific error;CH1: Segment id 1: Downloaded data length of 320 samples is not a multiple of 128."\n'

### free memory

In [194]:
write('trac1:del:all')

### alocar memória

In [178]:
write('trac1:def 1,4096,0')